# Mission to Mars - Web Scraper

In [24]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import requests
import time

In [25]:
#Setup scraper
url = "https://mars.nasa.gov/news"

!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url)

/usr/local/bin/chromedriver


## Scrape Article Data

In [26]:
#grab all the data
html = browser.html
soup = bs(html, 'html.parser')

In [27]:
#sift through the data
slide = soup.find('li', class_="slide")
news_title = slide.find('h3').text
news_p = slide.find('div', class_="article_teaser_body").text

## Scrape Image

In [29]:
#navigates to website and clicks to get more info
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

browser.click_link_by_partial_text('FULL IMAGE')

time.sleep(5)

browser.click_link_by_partial_text('more info')

In [30]:
#grab all the data
html = browser.html
soup = bs(html, 'html.parser')

In [31]:
#grab just the featured image
image = soup.find_all('div',class_="download_tiff")
featured_image_url = 'https:' + image[1].a['href']

## Scrape Mars Tweet

In [32]:
#goes to twitter and grabs data
url= "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html= browser.html
soup = bs(html, 'html.parser')

In [33]:
#grabs first tweet
mars_weather = soup.find('p', class_="tweet-text").text

## Scrap Mars Facts

In [34]:
#grab table with pandas
url= "http://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[0]

In [35]:
#clean up table
df.set_index(0)

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [36]:
df.columns = ['','']
mars_facts = df.to_html()
mars_facts.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </t

## Scrape Hemispheres

In [37]:
#goes to hemisphere website
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [38]:
hemispheres = ["Cerberus", "Schiaparelli", "Syrtis","Valles"]
hemisphere_image_urls = []

for hemisphere in hemispheres:
    browser.click_link_by_partial_text(hemisphere)
    html = browser.html
    soup = bs(html, 'html.parser')
    urls = soup.find('div', class_="downloads").find_all('li')
    img_url = urls[1].a['href']
    title = soup.find('title').text.split(' Enhanced')[0]
    hemisphere_image_urls.append({'title':title,'img_url':img_url})
    browser.click_link_by_partial_text('Back')

In [40]:
browser.quit()